# Import Necessary Libraries

In [205]:
import math
import os
import random

import pandas as pd
import numpy as np

from random import sample
from tqdm import tqdm
from conlleval import evaluate as conllevaluate

# Hyper-Parameters

In [234]:
SEED = 42
EPOCHS = 3
EARLY_STOPPING_LIMIT = 3
DEBUG = True
TRAINING_SAMPLES = 300
DEV_SAMPLES = 100
TEST_SAMPLES = 100

# Special Tokens

In [236]:
START = "<START>"
STOP = "<STOP>"

# Set Seed to make the training reproducible

In [237]:
def make_reproducible(seed: int=42) -> None:
    """Set seed to make the training reproducible."""
    np.random.seed(seed)
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)

make_reproducible(SEED)

# Read Train, Dev and Test NER Data Splits

In [238]:
def read_data(filename) -> list:
    """
    Reads the CoNLL 2003 data into an array of dictionaries (a dictionary for each data point).
    :param filename: String
    :return: Array of dictionaries.  Each dictionary has the format returned by the make_data_point function.
    """
    data = list()
    with open(filename, "r") as f:
        sent = list()
        for line in f.readlines():
            if line.strip():
                sent.append(line)
            else:
                data.append(make_data_point(sent))
                sent = list()
        data.append(make_data_point(sent))

    return data


def make_data_point(sent) -> dict:
    """
        Creates a dictionary from String to an Array of Strings representing the data.  The dictionary items are:
        dic['tokens'] = Tokens padded with <START> and <STOP>
        dic['pos'] = POS tags padded with <START> and <STOP>
        dic['NP_chunk'] = Tags indicating noun phrase chunks, padded with <START> and <STOP> (but will not use)
        dic['gold_tags'] = The gold tags padded with <START> and <STOP>
    :param sent: String.  The input CoNLL format string
    :return: Dict from String to Array of Strings.
    """
    dic = dict()
    sent = [s.strip().split() for s in sent]
    dic["tokens"] = [START] + [s[0] for s in sent] + [STOP]
    dic["pos"] = [START] + [s[1] for s in sent] + [STOP]
    dic["NP_chunk"] = [START] + [s[2] for s in sent] + [STOP]
    dic["gold_tags"] = [START] + [s[3] for s in sent] + [STOP]
    return dic


def read_gazetteer() -> list:
    data = list()
    with open("gazetteer.txt", "r") as f:
        for line in f.readlines():
            data += line.split()[1:]
    return data

datapath = "./data"
gazetteer = read_gazetteer()

train_data = read_data(os.path.join(datapath, "ner.train"))
dev_data = read_data(os.path.join(datapath, "ner.dev"))
test_data = read_data(os.path.join(datapath, "ner.test"))

# Tagset

In [239]:
tagset = [
    # Person (Begin, Inside)
    "B-PER", "I-PER",
    # Location (Begin, Inside)
    "B-LOC", "I-LOC",
    # Organization (Begin, Inside)
    "B-ORG", "I-ORG",
    # Miscelleneous (Begin, Inside)
    "B-MISC", "I-MISC",
    # Outside
    "O"
]

# Debug on subset of data

In [240]:
if DEBUG:
    train_data = sample(train_data, TRAINING_SAMPLES)
    dev_data = sample(dev_data, DEV_SAMPLES)
    test_data = sample(test_data, TEST_SAMPLES)
    
training_size = len(train_data)

In [241]:
class FeatureVector(object):
    def __init__(self, fdict):
        self.fdict = fdict

    def times_plus_equal(self, scalar, v2) -> None:
        """
        self += scalar * v2
        :param scalar: Double
        :param v2: FeatureVector
        :return: None
        """
        for key, value in v2.fdict.items():
            self.fdict[key] = scalar * value + self.fdict.get(key, 0)

    def dot_product(self, v2) -> int:
        """
        Computes the dot product between self and v2.  It is more efficient for v2 to be the smaller vector (fewer
        non-zero entries).
        :param v2: FeatureVector
        :return: Int
        """
        retval = 0
        for key, value in v2.fdict.items():
            retval += value * self.fdict.get(key, 0)
        return retval

    def square(self):
        retvector = FeatureVector({})
        for key, value in self.fdict.items():
            val_sq = value * value
            retvector.fdict[key] = val_sq
        return retvector

    def square_root(self):
        retvector = FeatureVector({})
        for key, value in self.fdict.items():
            val_sq = math.sqrt(value)
            retvector.fdict[key] = val_sq
        return retvector

    def divide(self, v2):
        retvector = FeatureVector({})
        for key, value in v2.fdict.items():
            if value == 0:
                retvector.fdict[key] = 0
            else:
                retvector.fdict[key] = self.fdict.get(key, 0) / value
        return retvector

    def write_to_file(self, filename) -> None:
        """
        Writes the feature vector to a file.
        :param filename: String
        :return: None
        """ 
        print("Writing to " + filename)
        path = os.path.join("./reports", filename)
        with open(path, "w", encoding="utf-8") as f:
            features = [(k, v) for k, v in self.fdict.items()]
            features.sort(key=lambda feature: feature[1], reverse=True)
            for key, value in features:
                f.write(f"{key} {value}\n")

    def read_from_file(self, filename) -> None:
        """
        Reads a feature vector from a file.
        :param filename: String
        :return: None
        """
        self.fdict = dict()
        with open(filename, "r") as f:
            for line in f.readlines():
                txt = line.split()
                self.fdict[txt[0]] = float(txt[1])


class Features(object):
    def __init__(self, inputs, feature_names):
        """
        Creates a Features object
        :param inputs: Dictionary from String to an Array of Strings.
            Created in the make_data_point function.
            inputs['tokens'] = Tokens padded with <START> and <STOP>
            inputs['pos'] = POS tags padded with <START> and <STOP>
            inputs['NP_chunk'] = Tags indicating noun phrase chunks, padded with <START> and <STOP>
            inputs['gold_tags'] = DON'T USE! The gold tags padded with <START> and <STOP>
        :param feature_names: Array of Strings.  The list of features to compute.
        """
        self.feature_names = feature_names
        self.inputs = inputs

    def compute_features(self, cur_tag, pre_tag, i):
        """
        Computes the local features for the current tag, the previous tag, and position i
        :param cur_tag: String.  The current tag.
        :param pre_tag: String.  The previous tag.
        :param i: Int. The position
        :return: FeatureVector
        """
        feats = FeatureVector({})
        cur_word = self.inputs["tokens"][i]
        pos_tag = self.inputs["pos"][i]
        is_last = len(self.inputs["tokens"]) - 1 == i
        # Feature-1: Current word(Wi)
        if "current_word" in self.feature_names:
            key = f"Wi={cur_word}+Ti={cur_tag}"
            add_features(feats, key)
        # Feature-2: Previous Tag(Ti-1)
        if "prev_tag" in self.feature_names:
            key = f"Ti-1={pre_tag}+Ti={cur_tag}"
            add_features(feats, key)
        # Feature-3: Lowercased Word(Oi)
        if "lowercase" in self.feature_names:
            key = f"Oi={cur_word.lower()}+Ti={cur_tag}"
            add_features(feats, key)
        # Feature-4: Current POS Tag(Pi)
        if "pos_tag" in self.feature_names:
            key = f"Pi={pos_tag}+Ti={cur_tag}"
            add_features(feats, key)
        # Feature-5: Shape of Current Word(Si)
        if "word_shape" in self.feature_names:
            word_shape = get_word_shape(cur_word)
            key = f"Si={word_shape}+Ti={cur_tag}"
            add_features(feats, key)
        # Feature-6: (1-4 for prev + for next)
        if "feats_prev_and_next" in self.feature_names:
            prev_word = self.inputs["tokens"][i - 1]
            prev_pos = self.inputs["pos"][i - 1]
            prev_1 = f"Wi-1={prev_word}+Ti={cur_tag}"
            prev_3 = f"Oi-1={prev_word.lower()}+Ti={cur_tag}"
            prev_4 = f"Pi-1={prev_pos}+Ti={cur_tag}"
            add_features(feats, prev_1)
            add_features(feats, prev_3)
            add_features(feats, prev_4)
            if not is_last:
                next_word = self.inputs["tokens"][i + 1]
                next_pos = self.inputs["pos"][i + 1]
                next_1 = f"Wi+1={next_word}+Ti={cur_tag}"
                next_3 = f"Oi+1={next_word.lower()}+Ti={cur_tag}"
                next_4 = f"Pi+1={next_pos}+Ti={cur_tag}"
                add_features(feats, next_1)
                add_features(feats, next_3)
                add_features(feats, next_4)
        # Feature-7: 1,3,4 conjoined with Previous Tag (pre_tag)
        if "feat_conjoined" in self.feature_names:
            conjoined_1 = f"Wi={cur_word}+Ti-1={pre_tag}+Ti={cur_tag}"
            conjoined_3 = f"Oi={cur_word.lower()}+Ti-1={pre_tag}+Ti={cur_tag}"
            conjoined_4 = f"Pi={pos_tag}+Ti-1={pre_tag}+Ti={cur_tag}"
            add_features(feats, conjoined_1)
            add_features(feats, conjoined_3)
            add_features(feats, conjoined_4)
        # Feature-8: Prefix for Current word with lenhth k where k=1,2,3,4
        if "prefix_k" in self.feature_names:
            for k in range(4):
                if k > len(cur_word):
                    break
                prefix = cur_word[: k + 1]
                key = f"PREi={prefix}+Ti={cur_tag}"
                add_features(feats, key)
        # Feature-9: Gazetteer (GAZi)
        if "gazetteer" in self.feature_names:
            key = f"GAZi={is_gazetteer(cur_word)}+Ti={cur_tag}"
            add_features(feats, key)
        # Feature-10: Is capital (CAPi)
        if "capital" in self.feature_names:
            key = f"CAPi={is_capital(cur_word)}+Ti={cur_tag}"
            add_features(feats, key)
        # Feature-11: Position of the current word (indexed from 1)
        if "position" in self.feature_names:
            key = f"POSi={i+1}+Ti={cur_tag}"
            add_features(feats, key)
        return feats


def add_features(feats, key) -> None:
    feats.times_plus_equal(1, FeatureVector({key: 1}))


def get_word_shape(word):
    shape = ""
    for c in word:
        shape += get_char_shape(c)
    return shape


def get_char_shape(char):
    encoding = ord(char)
    if encoding >= ord("a") and encoding <= ord("z"):
        return "a"
    if encoding >= ord("A") and encoding <= ord("Z"):
        return "A"
    if encoding >= ord("0") and encoding <= ord("9"):
        return "d"
    return char


def is_gazetteer(word) -> str:
    if word in gazetteer:
        return "True"
    return "False"


def is_capital(word) -> str:
    if len(word) == 0:
        return "False"
    c = ord(word[0])
    if c >= ord("A") and c <= ord("Z"):
        return "True"
    return "False"


def compute_features(tag_seq, input_length, features) -> FeatureVector:
    """
    Compute f(xi, yi)
    :param tag_seq: [tags] already padded with <START> and <STOP>
    :param input_length: input length including the padding <START> and <STOP>
    :param features: func from token index to FeatureVector
    :return:
    """
    feats = FeatureVector({})
    for i in range(1, input_length):
        feats.times_plus_equal(1, features.compute_features(tag_seq[i], tag_seq[i - 1], i))
    return feats

    # Examples from class (from slides Jan 15, slide 18):
    # x = will to fight
    # y = NN TO VB
    # features(x,y) =
    #  {"wi=will^yi=NN": 1, // "wi="+current_word+"^yi="+current_tag
    # "yi-1=START^yi=NN": 1,
    # "ti=to+^yi=TO": 1,
    # "yi-1=NN+yi=TO": 1,
    # "xi=fight^yi=VB": 1,
    # "yi-1=TO^yi=VB": 1}

    # x = will to fight
    # y = NN TO VBD
    # features(x,y)=
    # {"wi=will^yi=NN": 1,
    # "yi-1=START^yi=NN": 1,
    # "ti=to+^yi=TO": 1,
    # "yi-1=NN+yi=TO": 1,
    # "xi=fight^yi=VBD": 1,
    # "yi-1=TO^yi=VBD": 1}

# Feature Set

In [242]:
# Limited Feature Set -> Features 1-4
limited_feature_set = ["current_word", "prev_tag", "lowercase", "pos_tag"]
# Full Feature Set -> Features 1-4 + 5-11
full_feature_set = limited_feature_set + ["word_shape", "feats_prev_and_next", "feat_conjoined", "prefix_k", "gazetteer", "capital", "position"]

# Viterbi Decoding

In [243]:
def backtrack(viterbi_matrix, tagset, max_tag) -> list:
    tags = list()
    for k in reversed(range(len(viterbi_matrix))):
        max_tag_idx = tagset.index(max_tag)
        viterbi_list = viterbi_matrix[k]
        max_tag, _ = viterbi_list[max_tag_idx]
        tags = [max_tag] + tags
    return tags


def decode(input_len, tagset, score_func) -> list:
    """Viterbi Decoding to find the Best Tag Sequence"""
    tags = list()
    viterbi_matrix = list()
    # Initial Step ,i.e. t=1
    initial_list = list()
    for tag in tagset:
        score = score_func(tag, START, 1)
        initial_list.append((START, score))
    viterbi_matrix.append(initial_list)
    # Recursion Step ,i.e. t=2,3,4,...
    for t in range(2, input_len - 1):
        viterbi_list = list()
        for tag in tagset:
            max_tag = None
            max_score = float("-inf")
            for prev_tag in tagset:
                last_viterbi_list = viterbi_matrix[t - 2]
                prev_tag_idx = tagset.index(prev_tag)
                last_score = last_viterbi_list[prev_tag_idx][1]
                score = score_func(tag, prev_tag, t) + last_score
                if score > max_score:
                    max_score = score
                    max_tag = prev_tag
            viterbi_list.append((max_tag, score))
        viterbi_matrix.append(viterbi_list)
    # Termination Step
    tags = [STOP] + tags
    # Calculate the score for the max_tag
    last_viterbi_list = list()
    for tag in tagset:
        stop_score = score_func(STOP, tag, input_len - 1)
        prev_score = viterbi_matrix[-1][tagset.index(tag)][1]
        score = stop_score + prev_score
        last_viterbi_list.append((tag, score))
    max_tag, _ = max(last_viterbi_list, key=lambda tuple: tuple[1])
    tags = backtrack(viterbi_matrix, tagset, max_tag) + [max_tag] + tags
    return tags

In [244]:
def predict(inputs, input_len, parameters, feature_names, tagset, score_func):
    features = Features(inputs, feature_names)
    gold_labels = inputs["gold_tags"]
    score = score_func(gold_labels, parameters, features)
    return decode(input_len, tagset, score)


def write_predictions(out_filename, all_inputs, parameters, feature_names, tagset, score_func):
    """
    Writes the predictions on all_inputs to out_filename, in CoNLL 2003 evaluation format.
    Each line is token, pos, NP_chuck_tag, gold_tag, predicted_tag (separated by spaces)
    Sentences are separated by a newline
    The file can be evaluated using the command: python conlleval.py < out_file
    :param out_filename: filename of the output
    :param all_inputs:
    :param parameters:
    :param feature_names:
    :param tagset:
    :return:
    """
    pred_dir = "./predictions"
    if not os.path.exists(pred_dir):
        os.makedirs(pred_dir)
    path = os.path.join(pred_dir, out_filename)
    with open(path, "w", encoding="utf-8") as f:
        for inputs in all_inputs:
            input_len = len(inputs["tokens"])
            tag_seq = predict(inputs, input_len, parameters, feature_names, tagset, score_func)
            for i, tag in enumerate(tag_seq[1:-1]):
                f.write(" ".join([inputs["tokens"][i + 1], inputs["pos"][i + 1], inputs["NP_chunk"][i + 1], inputs["gold_tags"][i + 1], tag]) + "\n")
            f.write("\n")


def compute_score(tag_seq, input_length, score):
    """
    Computes the total score of a tag sequence
    :param tag_seq: Array of String of length input_length. The tag sequence including <START> and <STOP>
    :param input_length: Int. input length including the padding <START> and <STOP>
    :param score: function from current_tag (string), previous_tag (string), i (int) to the score.  i=0 points to
        <START> and i=1 points to the first token. i=input_length-1 points to <STOP>
    :return:
    """
    total_score = 0
    for i in range(1, input_length):
        total_score += score(tag_seq[i], tag_seq[i - 1], i)
    return total_score


def test_decoder() -> None:
    # See https://classes.soe.ucsc.edu/nlp202/Winter21/assignments/A1_Debug_Example.pdf

    tagset = ["NN", "VB"]  # make up our own tagset

    def score_wrap(cur_tag, pre_tag, i):
        retval = score(cur_tag, pre_tag, i)
        print(
            "Score("
            + cur_tag
            + ","
            + pre_tag
            + ","
            + str(i)
            + ") returning "
            + str(retval)
        )
        return retval

    def score(cur_tag, pre_tag, i):
        if i == 0:
            print(
                "ERROR: Don't call score for i = 0 (that points to <START>, with nothing before it)"
            )
        if i == 1:
            if pre_tag != "<START>":
                print(
                    "ERROR: Previous tag should be <START> for i = 1. Previous tag = "
                    + pre_tag
                )
            if cur_tag == "NN":
                return 6
            if cur_tag == "VB":
                return 4
        if i == 2:
            if cur_tag == "NN" and pre_tag == "NN":
                return 4
            if cur_tag == "NN" and pre_tag == "VB":
                return 9
            if cur_tag == "VB" and pre_tag == "NN":
                return 5
            if cur_tag == "VB" and pre_tag == "VB":
                return 0
        if i == 3:
            if cur_tag != "<STOP>":
                print(
                    "ERROR: Current tag at i = 3 should be <STOP>. Current tag = "
                    + cur_tag
                )
            if pre_tag == "NN":
                return 1
            if pre_tag == "VB":
                return 1

    predicted_tag_seq = decode(4, tagset, score_wrap)
    print("Predicted tag sequence should be = <START> VB NN <STOP>")
    print("Predicted tag sequence = " + " ".join(predicted_tag_seq))
    print(
        "Score of ['<START>','VB','NN','<STOP>'] = "
        + str(compute_score(["<START>", "VB", "NN", "<STOP>"], 4, score))
    )
    print("Max score should be = 14")
    print("Max score = " + str(compute_score(predicted_tag_seq, 4, score)))

In [245]:
def optimizer(update_func="ssgd", l2_lambda=0.01):
    sum_accumulator = FeatureVector({})
    def adagrad(i, gradient, parameters, step_size):
        grad = gradient(i)
        sum_accumulator.times_plus_equal(1, grad.square())
        parameters.times_plus_equal(-step_size, grad.divide(sum_accumulator.square_root()))
        return parameters

    def ssgd(i, gradient, parameters, step_size):
        grad = gradient(i)
        parameters.times_plus_equal(-step_size, grad)
        return parameters

    def l2_regularizer(i, gradient, parameters, step_size):
        grad = gradient(i)
        regularizer = FeatureVector({})
        regularizer.times_plus_equal(l2_lambda, parameters)
        parameters.times_plus_equal(-step_size, grad)
        parameters.times_plus_equal(-step_size, regularizer)
        return parameters

    update = ssgd
    if update_func == "adagrad":
        update = adagrad
    elif update_func == "l2_regularizer":
        update = l2_regularizer

    def optimizer_func(training_size, epochs, gradient, parameters, training_observer, step_size=1):
        no_improve_count = 0
        best_params = parameters
        max_score = float("-inf")
        for epoch in range(epochs):
            for i in tqdm(range(training_size), desc="Training...", colour="red"):
                parameters = update(i, gradient, parameters, step_size)
            # Evaluating on Dev Set
            cur_score = training_observer(epoch, parameters)
            cur_score = round(cur_score, 4)
            print(f"Epoch {epoch+1:02} -> F1-Score: {cur_score}")
            # Early Stopping
            if cur_score >= max_score:
                best_params = FeatureVector({})
                best_params.times_plus_equal(1, parameters)
                max_score = cur_score
                no_improve_count = 0
            else:
                no_improve_count += 1
            if no_improve_count > EARLY_STOPPING_LIMIT:
                return best_params
        return best_params
    return optimizer_func


def hamming_loss(loss_val=10, penalty=0):
    def loss(gold, pred):
        result = loss_val
        if penalty > 0:
            if gold != "O" and pred == "O":
                result = penalty * result
        return result if gold != pred else 0
    return loss


def svm_with_cost_func(cost_func):
    def score(gold_labels, parameters, features):
        return svm_score(gold_labels, parameters, features, cost_func=cost_func)
    return score


def perceptron_score(gold_labels, parameters, features):
    def score(cur_tag, pre_tag, i):
        return parameters.dot_product(
            features.compute_features(cur_tag, pre_tag, i))
    return score


def svm_score(gold_labels, parameters, features, cost_func=hamming_loss()):
    def score(cur_tag, pre_tag, i):
        cost_val = cost_func(gold_labels[i], cur_tag)
        cur_score = parameters.dot_product(features.compute_features(cur_tag, pre_tag, i))
        return cur_score + cost_val
    return score


def get_gradient(data, feature_names, tagset, parameters, score_func):

    def subgradient(i):
        inputs = data[i]
        input_len = len(inputs["tokens"])
        gold_labels = inputs["gold_tags"]
        features = Features(inputs, feature_names)
        score = score_func(gold_labels, parameters, features)
        # Viterbi Decoding to get the Predicted Tags
        tags = decode(input_len, tagset, score)
        fvector = compute_features(tags, input_len, features)
        fvector.times_plus_equal(-1, compute_features(gold_labels, input_len, features))
        return fvector

    return subgradient

In [246]:
def evaluate(data, parameters, feature_names, tagset, score_func, verbose=False):
    """Calculate Precision, Recall, and F1-Score on the given data."""
    all_gold_tags = list()
    all_predicted_tags = list()
    for inputs in tqdm(data, desc="Evaluating...", colour="green"):
        all_gold_tags.extend(inputs["gold_tags"][1:-1])
        input_len = len(inputs["tokens"])
        all_predicted_tags.extend(predict(inputs,input_len,parameters,feature_names,tagset,score_func)[1:-1])
    return conllevaluate(all_gold_tags, all_predicted_tags, verbose)


def write_reports(reports, filename, columns) -> None:
    report_map = dict()
    for i, column in enumerate(columns):
        values = list()
        for report in reports:
            value = report[i]
            values.append(value)
        report_map[column] = values
    
    reports_dir = "./reports"
    if not os.path.exists(reports_dir):
        os.makedirs(reports_dir)
    path = os.path.join(reports_dir, filename)
    report_df = pd.DataFrame(data=report_map)
    report_df.to_csv(path, index=False)

In [247]:
def train(data, feature_names, tagset, epochs, optimizer, score_func=perceptron_score, step_size=1):
    """Train the Model on the given data."""
    parameters = FeatureVector({})
    gradient = get_gradient(data, feature_names, tagset, parameters, score_func)
    
    def training_observer(epoch, parameters):
        """Evaluates the parameters on the Dev Set and returns the F1-Score."""
        (precision, recall, f1) = evaluate(dev_data, parameters, feature_names, tagset, score_func)
        return f1
    
    return optimizer(training_size, epochs, gradient, parameters, training_observer, step_size=step_size)

In [249]:
def eval_and_test(parameters, feature_names, score, name) -> None:
    report_cols = ["precision", "recall", "f1"]
    # Dev Report Generation
    report = evaluate(dev_data, parameters, feature_names, tagset, score)
    dev_precision, dev_recall, dev_f1 = report
    print(f"DEV SET | Precision: {dev_precision:.4f} | Recall: {dev_recall:.4f} | F-1 Score: {dev_f1:.4f}")
    write_predictions(f"{name}.dev.pred", dev_data, parameters, feature_names, tagset, score)
    write_reports([list(report)], f"{name}.dev.report", report_cols)
    # Test Report Generation
    report = evaluate(test_data, parameters, feature_names, tagset, score)
    test_precision, test_recall, test_f1 = report
    print(f"TEST SET | Precision: {test_precision:.4f} | Recall: {test_recall:.4f} | F-1 Score: {test_f1:.4f}")
    write_predictions(f"{name}.test.pred", test_data, parameters, feature_names, tagset, score)
    write_reports([list(report)], f"{name}.test.report", report_cols)

# Header Printer Helper Function

In [250]:
def header_printer(name: str) -> None:
    print("*" * 100)
    print(name)
    print("*" * 100)

In [251]:
def structured_perceptron(feature_names, name, optimizer, write_params=False) -> None:
    header_printer(name)
    parameters = train(train_data, feature_names, tagset, epochs=EPOCHS, score_func=perceptron_score, optimizer=optimizer)
    eval_and_test(parameters, feature_names, perceptron_score, name=name)
    if write_params:
        parameters.write_to_file(f"{name}.parameters")

# SVM Configuration Parameters

In [252]:
def svm_config(): 
    step_sizes = [10, 50, 100]
    lambdas = [5e-4, 1e-4, 1e-3]
    cols = ["step_size", 
            "l2_lambda", 
            "precision", 
            "recall", 
            "f1"]
    return step_sizes, lambdas, cols

In [253]:
def structured_svm(cost, feature_names, name) -> None:
    header_printer(name)
    config = svm_config()
    score = svm_with_cost_func(cost)
    best_f1_score = float("-inf")
    best_parameters = FeatureVector({})
    reports = list()
    for step_size in config[0]:
        for l2_lambda in config[1]:
            print(f"Step-Size: {step_size} | Lambda: {l2_lambda}")
            parameters = train(train_data, feature_names, tagset, epochs=1, step_size=step_size, score_func=score, optimizer=optimizer(update_func="l2_regularizer", l2_lambda=l2_lambda))
            precision, recall, f1 = evaluate(dev_data, parameters, feature_names, tagset, score)
            print(f"Tuning Values: Precision: {precision:.4f} | Recall: {recall:.4f} | F-1 Score: {f1:.4f}\n")
            reports.append([step_size, l2_lambda, precision, recall, f1])
            if f1 > best_f1_score:
                tuned_step_size = step_size
                tuned_l2_lambda = l2_lambda
                best_parameters = FeatureVector({})
                best_parameters.times_plus_equal(1, parameters)
                best_f1_score = f1
    print(f"\nBEST CONFIG: Step-Size: {tuned_step_size} | Lambda: {tuned_l2_lambda}\n")
    write_reports(reports, f"{name}.tuning.report", config[2])
    eval_and_test(best_parameters, full_feature_set, score, name=name)

# Limited Feature Set: Perceptron SSGD

In [224]:
structured_perceptron(feature_names=limited_feature_set, name="lfs_perceptron_ssgd", optimizer=optimizer(), write_params=True)

****************************************************************************************************
lfs_perceptron_ssgd
****************************************************************************************************


Evaluating...: 100%|██████████| 50/50 [00:00<00:00, 269.93it/s]


Epoch 01 -> F1-Score: 13.3333


Evaluating...: 100%|██████████| 50/50 [00:00<00:00, 266.19it/s]


Epoch 02 -> F1-Score: 44.8276


...

Epoch 03 -> F1-Score: 38.9189


Evaluating...: 100%|██████████| 50/50 [00:00<00:00, 272.06it/s]


DEV SET | Precision: 50.6494 | Recall: 40.2062 | F-1 Score: 44.8276


Evaluating...: 100%|██████████| 50/50 [00:00<00:00, 297.13it/s]


TEST SET | Precision: 45.0000 | Recall: 42.3529 | F-1 Score: 43.6364
Writing to lfs_perceptron_ssgd.parameters


# Full Feature Set: Perceptron SSGD

In [225]:
structured_perceptron(feature_names=full_feature_set, name="ffs_perceptron_ssgd", optimizer=optimizer(), write_params=True)

****************************************************************************************************
ffs_perceptron_ssgd
****************************************************************************************************


Evaluating...: 100%|██████████| 50/50 [00:22<00:00,  2.18it/s]


Epoch 01 -> F1-Score: 40.9357


Evaluating...: 100%|██████████| 50/50 [00:22<00:00,  2.22it/s]


Epoch 02 -> F1-Score: 52.0


...

Epoch 03 -> F1-Score: 60.9626


Evaluating...: 100%|██████████| 50/50 [00:23<00:00,  2.16it/s]


DEV SET | Precision: 63.3333 | Recall: 58.7629 | F-1 Score: 60.9626


Evaluating...: 100%|██████████| 50/50 [00:21<00:00,  2.29it/s]


TEST SET | Precision: 59.0361 | Recall: 57.6471 | F-1 Score: 58.3333
Writing to ffs_perceptron_ssgd.parameters


# Full Feature Set: Perceptron Adagrad

In [227]:
structured_perceptron(feature_names=full_feature_set, name="ffs_perceptron_adagrad", optimizer=optimizer(update_func="adagrad"))

****************************************************************************************************
ffs_perceptron_adagrad
****************************************************************************************************


Evaluating...: 100%|██████████| 50/50 [00:23<00:00,  2.13it/s]


Epoch 01 -> F1-Score: 57.2917


Evaluating...: 100%|██████████| 50/50 [00:23<00:00,  2.15it/s]


Epoch 02 -> F1-Score: 59.893


...

Epoch 03 -> F1-Score: 67.3684


Evaluating...: 100%|██████████| 50/50 [00:22<00:00,  2.17it/s]


DEV SET | Precision: 68.8172 | Recall: 65.9794 | F-1 Score: 67.3684


Evaluating...: 100%|██████████| 50/50 [00:21<00:00,  2.31it/s]


TEST SET | Precision: 65.8824 | Recall: 65.8824 | F-1 Score: 65.8824


# Full Feature Set: SVM SSGD

In [254]:
structured_svm(hamming_loss(), feature_names=full_feature_set, name="ffs_svm_ssgd")

****************************************************************************************************
ffs_svm_ssgd
****************************************************************************************************
Step-Size: 10 | Lambda: 0.0005


Training...:  20%|█▉        | 59/300 [00:25<01:45,  2.29it/s]


KeyboardInterrupt: 

# Full Feature Set: Modified SVM SSGD

In [ ]:
structured_svm(hamming_loss(penalty=30), feature_names=full_feature_set, name="ffs_modified_svm_ssgd")

****************************************************************************************************
ffs_modified_svm_ssgd
****************************************************************************************************
Step-Size: 10 | Lambda: 0.0005


Evaluating...: 100%|██████████| 50/50 [00:22<00:00,  2.22it/s]


Epoch 01 -> F1-Score: 18.75


Evaluating...: 100%|██████████| 50/50 [00:22<00:00,  2.24it/s]


Tuning Values: Precision: 23.8095 | Recall: 15.4639 | F-1 Score: 18.7500

Step-Size: 10 | Lambda: 0.0001


...

Tuning Values: Precision: 100.0000 | Recall: 3.0928 | F-1 Score: 6.0000


BEST CONFIG: Step-Size: 10 | Lambda: 0.0001



Evaluating...: 100%|██████████| 50/50 [00:22<00:00,  2.21it/s]


DEV SET | Precision: 49.3333 | Recall: 38.1443 | F-1 Score: 43.0233


Evaluating...: 100%|██████████| 50/50 [00:21<00:00,  2.35it/s]


TEST SET | Precision: 48.5714 | Recall: 40.0000 | F-1 Score: 43.8710
